In [ ]:
'''Importing modules'''
import pandas
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
import json
from sklearn.cross_validation import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin, TransformerMixin
from sklearn.linear_model import Ridge, LinearRegression, Lasso, SGDRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import pickle
import dill
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer, TfidfTransformer
from nltk.corpus import stopwords
from sys import getsizeof
import re
from sklearn.pipeline import Pipeline, FeatureUnion

'''Initializing variables and empty lists'''
nlp_dict_list = []
ml_dict_list = []
nlp_iter_lines = 0
ml_iter_lines = 0
iter_lower_split = 0
text_list_lower_split = []
business_id_list = []
a = 0
b = 0
text_list = []

'''Opening txt file containing Yelp review data and reading all lines'''
nlp_yelp_dataset_txt = open('yelp_train_academic_dataset_review.txt', 'rb+')
nlp_raw_lines = nlp_yelp_dataset_txt.readlines()
ml_yelp_dataset_txt = open('../Assignment_3_ML/yelp_dataset_Raw.txt', 'rb+')
ml_raw_lines = ml_yelp_dataset_txt.readlines()

'''Writing nlp raw_lines to an empty list using json.loads to parse'''
while nlp_iter_lines < 1000: #len(nlp_raw_lines):
    if len(nlp_raw_lines[nlp_iter_lines]) > 0:
        nlp_dict_list.append(json.loads(nlp_raw_lines[nlp_iter_lines]))
    nlp_iter_lines += 1
    
'''Writing ml raw_lines to an empty list using json.loads to parse'''
while ml_iter_lines < 1000: #len(ml_raw_lines):
    if len(ml_raw_lines[ml_iter_lines]) > 0:
        ml_dict_list.append(json.loads(ml_raw_lines[ml_iter_lines]))
    ml_iter_lines += 1
    
'''Initializing list with Yelp data into a pandas data frame for manipulation'''
nlp_data_frame = pandas.DataFrame(nlp_dict_list).to_dict()
ml_data_frame = pandas.DataFrame(ml_dict_list).to_dict()

'''Closing txt files'''
nlp_yelp_dataset_txt.close()
ml_yelp_dataset_txt.close()

'''Initializing stars and text lists'''
nlp_stars_list = nlp_data_frame['stars'].values()
nlp_text_list = nlp_data_frame['text'].values()
nlp_business_id = nlp_data_frame['business_id'].values()
ml_business_id = ml_data_frame['business_id'].values()
ml_category = ml_data_frame['categories'].values()

'''Checking and matching the business id to restaurant categories'''
for index, categories in enumerate(ml_category):
    for inner_index, inner_categories in enumerate(categories):
        if inner_categories == 'Restaurants':
            business_id_list.append(ml_business_id[index])

while b < len(nlp_business_id):
    if [business_id for business_id in business_id_list if business_id in nlp_business_id[b]]:
        text_list.append(nlp_text_list[b])
    b += 1

'''Initializing stopwords and adding new stopwords to the list'''
stop_words = stopwords.words("english")
stop_words_additions = ['ll', 'adwmuxsza', 'zu', 'abc', 'aac', 'aardbark', 'aabc', 'aab', 'aaa', 'aa', 'that', 'youre', 'zzcrkebcrfxbb', 'zse', 'i\'m', 'he\'s', 'i\'ve', 'it\'s', 'id', 'im', 'hes', 'ive', 'its']
while a < len(stop_words_additions):
    stop_words.append(stop_words_additions[a])
    a += 1
    
'''Looping through text_list converting to lower case and splitting into individual terms'''
while iter_lower_split < len(text_list):
    text_list_lower = text_list[iter_lower_split].lower()
    text_list_split = text_list_lower.split()
    text_list_split = [w for w in text_list_split if not w in stop_words]
    text_list_split = (' '.join(text_list_split))
    text_list_split = re.findall('[a-z]{2,}', text_list_split)
    text_list_split = (' '.join(text_list_split))
    text_list_lower_split.append(text_list_split)
    iter_lower_split += 1
    
'''Initializing analytical model axes'''
X = text_list_lower_split
y = nlp_stars_list
    
count_vectorizer_unigram = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None)
count_vectorizer_bigram = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, ngram_range = (2, 2))

cv_unigram = count_vectorizer_unigram.fit_transform(X, y)
cv_unigram = np.asarray(cv_unigram.toarray())

cv_bigram = count_vectorizer_bigram.fit_transform(X, y)
cv_bigram = np.asarray(cv_bigram.toarray())

unigram_count = zip(count_vectorizer_unigram.get_feature_names(), cv_unigram.sum(axis = 0))
bigram_count = zip(count_vectorizer_bigram.get_feature_names(), cv_bigram.sum(axis = 0))

unigram_dict = {}
bigram_dict = {}

for unigram in unigram_count:
    unigram_dict.update({unigram[0]: unigram[1]})
    
for bigram in bigram_count:
    bigram_dict.update({bigram[0]: bigram[1]})

for bigrams in bigram_dict:
    bigrams_word  = bigrams.split()
    bigrams_count = bigram_dict[bigrams]
    score = (bigrams_count * 1.0) / (unigram_dict[bigrams_word[0]] * unigram_dict[bigrams_word[1]] + 15000)
    print str(bigrams) + '   -   ' + str(score)